In [9]:
import os
import json
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import LSTM, Dropout, Dense, Activation, Embedding

In [10]:
# Initialize paths for data input and weights output
data_dir = "../Data/"
data_file = "ONeill_trimmed_quarter.txt"
save_weights_dir = '../Trained_Weights/Weights_ONeill_quart_64b_128s_final/'
log_dir = "../Data/log.csv"
charToIndex_json = "char_to_index.json"

# Parameters
BATCH_SIZE = 64
SEQ_LENGTH = 128

In [11]:
def make_model(unique_chars):
    model = Sequential()
    
    # input shape is now (1,1) since we're only feeding the starting character - i.e. the starting note
    # for the generated sequence
    model.add(Embedding(input_dim = unique_chars, output_dim = 8, batch_input_shape = (1, 1))) 
  
    model.add(LSTM(256, return_sequences = True, stateful = True))
    model.add(Dropout(0.2))
    
    model.add(LSTM(256, return_sequences = True, stateful = True))
    model.add(Dropout(0.2))
    
    model.add(LSTM(256, stateful = True)) 
    model.add(Dropout(0.2))
    
    model.add((Dense(unique_chars)))
    model.add(Activation("softmax"))
    
    return model

In [12]:
def trim_sequence(generated_seq):
    print("\nRAW generated sequence: \n" + generated_seq)
    
    # the generated sequence ususally contains a few meaningless characters before a newline. Generally the rythm
    # and more cohesive notes are generated from the next line onward, so we remove the unnecessary characters.
    count = 0
    for char in generated_seq:
        count += 1
        if char == "\n" and generated_seq[count] == "\n":
            break
    seq_trimmed_before = generated_seq[count+1:]
    
    # The training data contains multiple songs, each separated by three newline characters. The model has learned this pattern and 
    # also adds three new line characters by itself. We would like to consider one song at time, so we ignore what follows after
    # these ending newlines.
    count = 0
    for char in seq_trimmed_before:
        count += 1
        if char == "\n" and seq_trimmed_before[count] == "\n":
            break
    seq_trimmed = seq_trimmed_before[:count]
    
    return seq_trimmed

In [21]:
def generate_sequence(epoch, initial_note, seq_length):
    file = open(os.path.join(data_dir, data_file), mode = 'r')
    data = file.read()
    file.close()
    # Load character mapping
    char_to_index = {char: x for (x, char) in enumerate(sorted(list(set(data))))}
    print("Unique characters in the training data = {}".format(len(char_to_index)))  
    #with open(os.path.join(data_dir, charToIndex_json)) as f:
    #    char_to_index = json.load(f)
    index_to_char = {x:char for char, x in char_to_index.items()}
    unique_chars = len(index_to_char)
    
    model = make_model(unique_chars)
    model.load_weights(save_weights_dir + "Weights_{}.h5".format(epoch))
    #model.summary()
    sequence_index = [initial_note]
    
    for _ in range(seq_length):
        batch = np.zeros((1, 1))
        batch[0, 0] = sequence_index[-1]
        
        # predict the probabilities for the next input character
        predicted_probs = model.predict_on_batch(batch).ravel()
        # randomly sample the character based on the probabilities from the network
        char_sample = np.random.choice(range(unique_chars), size = 1, p = predicted_probs)
        sequence_index.append(char_sample[0])
    
    # obtain a string of the notes corresponding to each generated character
    gen_sequence = ''.join(index_to_char[c] for c in sequence_index)
    trimmed = trim_sequence(gen_sequence)
    return trimmed

In [22]:
epoch = 80
initial_note = 42 # any integer between 0 and the number of uniqe characters in the data set
seq_length = 800 # shouldn't be below 300 in order to generate a valid sequence

music = generate_sequence(epoch, initial_note, seq_length)

print("\nTRIMMED generated sequence: \n")

print(music)

Unique characters in the training data = 92

RAW generated sequence: 
L#9E/2) (EA,).|
D2 A/2A/2 BA | F2 D(E/F/) GG | F2 D G3/2B/2 |
({d}A>G) (GA).A | {d}(Bcc) c(2-A/2B/2) | ccA G<A | G3 ||
(
c/2c/2) | efg ga | eef fdc | B((db/2f/2) gf | ({gf}cc).B | c3 ||


X: 24
T: The Black Slender Boy
M: 3/4
L: 1/8
B: "O'Neill's 28"
N: "Moderate"
N: "Collected by J.O'Neill"
N:"Mollected by F. O'Neill"
Z: 1997 by John Chambers <jc@trillian.mit.edu>
M: 3/4
L: 1/8
K:A
(d>c) \
| B2 (GG) (E>D) | (D>F) (G2 GG) | {A}(EG) .G.A .B.A A | (Bd) e2 (dd) |
| (e{cd}d>).A (GG) | (GG) (.G2 .G) | (Gd) (Be) | (de) d2 Bc | A4 (DA) |
| (Bc) (ee) (dc) | (AG) (EG) | (eg) e2 | (ff) (dc) | dB A,z \
| D2 (GA) | (A>A) (EE) | (EA) (Bd) | (de) e2 | (.d c)A F ||


X: 248
T: the HRameer Wose If mhe Ero
B: O'Neill's 299
N: "Moderate"
N: "Collected by J.O'Neill"
Z: 1997 by John Chambers <jc@trillian.mit.e

TRIMMED generated sequence: 


X: 24
T: The Black Slender Boy
M: 3/4
L: 1/8
B: "O'Neill's 28"
N: "Moderate"
N: "Collected by J.